# MULTI-AGENT SYSTEM

This system has 2-3 agents:
- coder, this is the one that has to generate a cypher query
- executor, this is the one that test the query on a neo4j database
- retriever, this is the agent that will pass the documents to be used by coder. It cannot be considered as a part of the system because it only initiate the chat with the documents.

In [17]:
from autogen import AssistantAgent, GroupChatManager, GroupChat, config_list_from_json
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent, TEXT_FORMATS
import os

from CypherExecutor import CypherCodeExecutor

In [18]:

config_list = config_list_from_json(env_or_file="CONFIG_LIST", filter_dict={"model": "gpt-4o"})
config_list[0]["api_key"] = os.environ.get("GITHUB_TOKEN")

llm_config = {"config_list": config_list, "temperature": 0.2}

prompt = """You are a data scientist that works with Cypher queries.
Your work is to translate the given answer as a Cypher query.

You have some rules to respect:
- You must use cypher language.
- You can use the pdf files. 
- Every query must be in a separate cypher format.
- You must be precise and explain everything, code outputs too.
- You will receive the code result when executor will execute it 

You can reply with 'TERMINATE' if:
- exit code is 0 and you explained the result
- exit code is 1 and the problem can't be fixed via query

Now i will give you some information about the database schema.
- nodes -
(:Movie), Describe a movie with a title and a plot. It may have the number of likes.
(:Person), Describe actors and directors. They have a name and a birthday.

- relationships -
(:Person) -[:ACTED_IN]-> (:Movie)

QUESTION IS:
{input_question}
"""

print(f'Accepted formats for "docs_path": \n{TEXT_FORMATS}')

Accepted formats for "docs_path": 
['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf']


In [19]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

doc_retriever = RetrieveUserProxyAgent(
    name="doc_retriever",
    is_termination_msg=termination_msg,
    max_consecutive_auto_reply=3,
    human_input_mode="NEVER",
    retrieve_config={
        "task": "code",
        "docs_path": "https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf",
        "model": config_list[0]["model"],
        "get_or_create": True,
        "customized_prompt": prompt
    },
    code_execution_config=False,
    description="Assistant who has extra content retrieval power for solving cypher queries."
)

coder      = AssistantAgent(
    name="coder",
    is_termination_msg=termination_msg,
    system_message=prompt,
    llm_config=llm_config,
)

executor   = AssistantAgent(
    name="executor",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    description= """Executor provides feedback based on the result of the query.""",
    code_execution_config={"executor": CypherCodeExecutor()}
    )


PROBLEM  = "Who acted in Pocahontas"

def _reset_agents():
    """
    This function reset all the agents used for the group chat.
    This should be used every time you start a new conversation.
    """
    doc_retriever.reset()
    coder.reset()
    executor.reset()

def state_transition(last_speaker, groupchat):
    """
    This function simply return the next speaker based on the last one.
    In this case we decided that doc_retriever should speak only to initiate chat
    and then the chat is from coder to executor in a sort of loop.
    """
    if   last_speaker is coder: return executor
    elif last_speaker is executor: return coder
    elif last_speaker is doc_retriever: return coder

def rag_chat():
    _reset_agents()
    groupchat = GroupChat(agents=[doc_retriever, coder, executor], messages=[], 
                          max_round=5, speaker_selection_method=state_transition)
    manager   = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    doc_retriever.initiate_chat(manager,message=doc_retriever.message_generator, problem=PROBLEM,n_results=3)


In [20]:
rag_chat()

2024-09-16 09:44:19,664 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs`.


Trying to create collection.


max_tokens is too small to fit a single line of text. Breaking this line:
	          ...
Failed to split docs with must_break_at_empty_line being True, set to False.
2024-09-16 09:44:23,829 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 140 chunks.


VectorDB returns doc_ids:  [['d7ee553c', '8b4016d0', '5a43ff8d']]
Adding content of doc d7ee553c to context.
Adding content of doc 8b4016d0 to context.
Adding content of doc 5a43ff8d to context.
doc_retriever (to chat_manager):

You are a data scientist that works with Cypher queries.
Your work is to translate the given answer as a Cypher query.

You have some rules to respect:
- You must use cypher language.
- You can use the pdf files. 
- Every query must be in a separate cypher format.
- You must be precise and explain everything, code outputs too.
- You will receive the code result when executor will execute it 

You can reply with 'TERMINATE' if:
- exit code is 0 and you explained the result
- exit code is 1 and the problem can't be fixed via query

Now i will give you some information about the database schema.
- nodes -
(:Movie), Describe a movie with a title and a plot. It may have the number of likes.
(:Person), Describe actors and directors. They have a name and a birthday.



[{'p.name': 'Mel Gibson'},
 {'p.name': 'David Ogden Stiers'},
 {'p.name': 'Irene Bedard'},
 {'p.name': 'Judy Kuhn'}]

executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: [{'p.name': 'Mel Gibson'}, {'p.name': 'David Ogden Stiers'}, {'p.name': 'Irene Bedard'}, {'p.name': 'Judy Kuhn'}]

--------------------------------------------------------------------------------

Next speaker: coder

coder (to chat_manager):

The query successfully retrieved the names of the actors who acted in the movie "Pocahontas". The result includes:

- Mel Gibson
- David Ogden Stiers
- Irene Bedard
- Judy Kuhn

These are the actors associated with the movie "Pocahontas" in the database.

TERMINATE

--------------------------------------------------------------------------------

Next speaker: executor

